# AlphaZero-like Chess Model Training

This notebook implements training for an AlphaZero-like chess model using PyTorch and Google Colab's GPU.

## 1. Setup Environment

First, let's ensure we have GPU access and install required packages.

In [ ]:
# Check if GPU is available
!nvidia-smi

In [ ]:
# Clone your repository (replace with your repo URL)
!git clone https://github.com/YOUR_USERNAME/AlphaZero-Chess.git
%cd AlphaZero-Chess

In [ ]:
# Install dependencies
!pip install -r requirements.txt

## 2. Import Dependencies and Setup

In [ ]:
import torch
import chess
from model import ChessNet, encode_board
from mcts import MCTS
from train import train_model

# Verify GPU availability
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')
if torch.cuda.is_available():
    print(f'GPU: {torch.cuda.get_device_name(0)}')
    print(f'Memory Available: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB')

## 3. Configure Training Parameters

Adjust these parameters based on your available compute resources:

In [ ]:
# Training configuration
config = {
    'num_iterations': 10,      # Number of training iterations
    'games_per_iteration': 20, # Self-play games per iteration
    'num_epochs': 5,          # Training epochs per iteration
    'batch_size': 64,         # Batch size for training
    'mcts_sims': 100         # MCTS simulations per move (reduced for Colab)
}

print('Training Configuration:')
for k, v in config.items():
    print(f'{k}: {v}')

## 4. Start Training

This will start the training process. The model will:
1. Play self-play games using MCTS
2. Collect training data
3. Train on the collected data
4. Save checkpoints after each iteration

In [ ]:
# Start training with configured parameters
train_model(
    num_iterations=config['num_iterations'],
    games_per_iteration=config['games_per_iteration'],
    num_epochs=config['num_epochs'],
    batch_size=config['batch_size']
)

## 5. Save and Download Model

After training completes, you can download the model checkpoints to your local machine.

In [ ]:
# Zip the model checkpoints
!zip -r model_checkpoints.zip model_checkpoint_*.pt

from google.colab import files
files.download('model_checkpoints.zip')

## 6. Test the Model (Optional)

You can test the trained model by playing a few moves:

In [ ]:
def test_model(model_path):
    # Load model
    model = ChessNet().to(device)
    checkpoint = torch.load(model_path)
    model.load_state_dict(checkpoint['model_state_dict'])
    model.eval()
    
    # Initialize board and MCTS
    board = chess.Board()
    mcts = MCTS(model, num_simulations=100, device=device)
    
    # Play a few moves
    for _ in range(5):
        if board.is_game_over():
            break
            
        # Get MCTS policy
        policy = mcts.search(board)
        
        # Select best move
        move_idx = policy.argmax().item()
        for move in board.legal_moves:
            if move_to_index(move) == move_idx:
                print(f'Move: {move.uci()}')
                board.push(move)
                print(board)
                print('\n')
                break

# Test the final model
test_model('model_checkpoint_10.pt')